# Employee Pipeline for Entirprise Internal CHatbot Step 1

This notebook contains the decision making/ evaluation aspects we took into consideration for different chunk sizes, chunk overlaps and top-ks


## 1. Importing libraries

In [2]:
#pip installing:
%pip install langchain
%pip install langchain_community
%pip install langchain_huggingface
%pip install langchain_pinecone
%pip install dotenv
%pip install pymupdf

#Pip installing for milvus:
%pip install -qU  langchain_milvus



import os
import langchain #its giving module not found error
import langchain_community
import langchain_huggingface
import langchain_pinecone
import dotenv

#Testing Miluvs imports:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from uuid import uuid4

#for timing the retrivals
import time




  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.8 MB/s eta 0:00:00


## Documentation/Helpful Links

Using RAG with langchain and FAST API:
https://www.youtube.com/watch?v=Arf7UwWjGyc

Building Milvus vector DBs with langchain:
https://python.langchain.com/docs/integrations/vectorstores/milvus/

timing python programs:
https://stackoverflow.com/questions/1557571/how-do-i-get-time-of-a-python-programs-execution


## Step 1: Determining Chunk size, chunk overlap and embedding dimensions:

Using milvus as a vector DB for testing purposes, will revert back to pinecone for the final product

## For Chunk Sizes:

First i read these article on choosing right chunk sizes:

https://www.llamaindex.ai/blog/evaluating-the-ideal-chunk-size-for-a-rag-system-using-llamaindex-6207e5d3fec5

https://www.mattambrogi.com/posts/chunk-size-matters/

then, i came to know 128 is the smallest chunk size (usually) so this can be one, then there is 256 (which is more common i believe) and for smaller chunk sizes, the issue is relevant response may not be in the top-2 or top-3 returns from vectordb, so in that case, we would need to set k's value more, however, the benefit being that the second article suggests that this is the most accurate chunk size. and for larger chunk sizes, the issue is of speed as larger chunks are slower to fetch. I now have a general understanding that smaller chunk sizes are beneficial. If we look at our dataset of RAG documents, the smallest document is "Employee Terminal Policy" in HR folder with content worth 1.5 pages, with 625 words.

For our personal evaluation, since we want to optimize for accuracy and relevant chunks without loosing out on speed, we will be testing top 3 most relevant retrievals against 3 categories:

#### Low: ( Chunk Size: 128)
#### medium: ( Chunk Size: 256)
#### high: ( Chunk Size: 1024)

I have decided to keep the chunk overlap at 10%, as per this question: https://learn.microsoft.com/en-us/answers/questions/1551865/how-do-you-set-document-chunk-length-and-overlap-w

The evaluation metric we will be using is a custom one, with questions from all 8 documents in the HR folder, and seeing how relevant the top 3 returned responses are to the query, and to which document they belong.



In [7]:
#Low Chunk Code
embeddings = HuggingFaceEmbeddings()
combined_docs = []
CHUNK_SIZE = 128
CHUNK_OVERLAP = 0.1
URI = "./Low_Chunk_Size.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

docs_to_load = ["Code-of-conduct.pdf", "Compensation-Benefits-Guide.pdf", "Employee Termination Policy.pdf", "Employee-Handbook.pdf", "Onboarding Manual.pdf", "employee-appraisal-form.pdf", "health-and-safety-guidelines.pdf", "remote-work-policy.pdf"]
for doc in docs_to_load:
  loader = PyMuPDFLoader(doc)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_SIZE*CHUNK_OVERLAP)
  docs = text_splitter.split_documents(documents)
  for j in range(len(docs)):
      combined_docs.append(docs[j])

vector_store_saved = Milvus.from_documents(
    combined_docs,
    embeddings,
    collection_name="Hr_Low_Chunk_Size",
    connection_args={"uri": URI},
)

vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="Hr_Low_Chunk_Size",
)

uuids = [str(uuid4()) for _ in range(len(combined_docs))]
vector_store.add_documents(documents=combined_docs, ids=uuids)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 0262309a92974677aca332d054240534
/usr/local/lib/python3.10/dist-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 0 of document health-and-safety-guidelines.pdf
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 3 of document health-and-safety-guidelines.pdf
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/document_loaders/parsers/pdf.py:29

* Page  3 
 
 
 
 
 
Dear Colleagues: 
 
Our Code of Business Conduct and Ethics for Employees is the most important document at 
Comerica.  It is the foundation on which all our business practices at Comerica are constructed and, 
for that reason, I consider it a critical one for each of us to read and understand. 
 
Our Code of Business Conduct and Ethics for Employees is a values-based document, rather than 
compliance-based, which means it goes beyond a simple listing of right and wrong.  As you read 
through, you will see that the Code of Business Conduct and Ethics for Employees explains in 
detail the ethical business practices and conduct that must govern our life here at Comerica. 
 
We are one of the leading financial institutions in the United States today.  There are many, many 
reasons for our success, but I believe a major reason is our integrity and trustworthiness - and that is 
what this Code of Business Conduct and Ethics for Employees is really all about. 
 
In the f

In [10]:
#This is from Code-Of_conduct
start_time = time.time()
results = vector_store.similarity_search(
    "What are the ethical business practices?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF CODE OF CONDUCT QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")
#This is from Compensation-Benefits Guide
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Medical Insurance & Surviving Spouse Coverage",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF COMPENSATIONS-BENEFITS GUIDE QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#This is from Employee Termination policy handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Voluntary Terminations",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE TERMINATION QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#this is from employee handbook, but conflicts with compensation-benefits, so we have to see if it is captured correctly from employee handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Compensation & development?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE HANDBOOK QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

QUERY EXECUTED IN --- 0.7881393432617188 seconds ---
RESULTS OF CODE OF CONDUCT QUERY: 0
CONTENT: Page  4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
SECTION 1 
 
ETHICAL BUSINESS PRACTICES 
 
• We should conduct our business in accordance with all material applicable laws, rules and 
regulations. 
 
• We should maintain the highest standards of ethical business conduct and integrity by: 
 
 Being fair and honest in all business dealings, including our professional relationships; 
 
 Properly maintaining all information and records, recognizing errors and, when an error is 
confirmed, promptly correcting it; and 
 
 Cooperating fully with all internal and external audits and investigations initiated or 
sanctioned by Comerica. 
 
• We must protect the confidentiality and privacy of confidential customer, shareholder, 
proprietary and third-party information and records. 
 
• We must make business decisions that align with Comerica’s risk appetite

## Low Chunk Size Observations:

so for query 1, the response 1 and 2 were highly relevant, but had some irrelevant data. response 3 was very poor, but for this query, all 3 chunks retrieved were from the correct file

for query 2, all 3 responses were highly relevant with response 1 and 2 being the most relevant. and all 3 were retrieved from the correct document

for query 3, response 1 and 2 were from employee handbook file, but were regarding employee terminations and response 3 was from the original employee terminations file, it can be inspected that the termination details are stronger in response 1 and 2. however, the "voluntary" terminations are more seen in response 3

for query 4, response 1 and 2 were irrelevant and from the wrong file (compensations was chosen as a misleading term in this example), however, response 3 was relevant and from the correct file.

## Reflection:
we need to parse and clean the data before turning it into embeddings (specifically, remove the contents.) furthermore, i think retrival and chunk size was fine for this one, khair, lets explore the other ones

In [ ]:
#Medium Chunk Code
embeddings = HuggingFaceEmbeddings()
combined_docs = []
CHUNK_SIZE = 256
CHUNK_OVERLAP = 0.1
URI = "./Medium_Chunk_Size.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

docs_to_load = ["Code-of-conduct.pdf", "Compensation-Benefits-Guide.pdf", "Employee Termination Policy.pdf", "Employee-Handbook.pdf", "Onboarding Manual.pdf", "employee-appraisal-form.pdf", "health-and-safety-guidelines.pdf", "remote-work-policy.pdf"]
for doc in docs_to_load:
  loader = PyMuPDFLoader(doc)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_SIZE*CHUNK_OVERLAP)
  docs = text_splitter.split_documents(documents)
  for j in range(len(docs)):
      combined_docs.append(docs[j])

vector_store_saved = Milvus.from_documents(
    combined_docs,
    embeddings,
    collection_name="Hr_Medium_Chunk_Size",
    connection_args={"uri": URI},
)

vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="Hr_Medium_Chunk_Size",
)

uuids = [str(uuid4()) for _ in range(len(combined_docs))]
vector_store.add_documents(documents=combined_docs, ids=uuids)

In [ ]:
#This is from Code-Of_conduct
start_time = time.time()
results = vector_store.similarity_search(
    "What are the ethical business practices?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF CODE OF CONDUCT QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")
#This is from Compensation-Benefits Guide
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Medical Insurance & Surviving Spouse Coverage",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF COMPENSATIONS-BENEFITS GUIDE QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#This is from Employee Termination policy handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Voluntary Terminations",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE TERMINATION QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#this is from employee handbook, but conflicts with compensation-benefits, so we have to see if it is captured correctly from employee handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Compensation & development?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE HANDBOOK QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

## Medium Chunk Size Observations:



## Reflections:


In [ ]:
#High Chunk Code
embeddings = HuggingFaceEmbeddings()
combined_docs = []
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 0.1
URI = "./High_Chunk_Size.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

docs_to_load = ["Code-of-conduct.pdf", "Compensation-Benefits-Guide.pdf", "Employee Termination Policy.pdf", "Employee-Handbook.pdf", "Onboarding Manual.pdf", "employee-appraisal-form.pdf", "health-and-safety-guidelines.pdf", "remote-work-policy.pdf"]
for doc in docs_to_load:
  loader = PyMuPDFLoader(doc)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_SIZE*CHUNK_OVERLAP)
  docs = text_splitter.split_documents(documents)
  for j in range(len(docs)):
      combined_docs.append(docs[j])

vector_store_saved = Milvus.from_documents(
    combined_docs,
    embeddings,
    collection_name="Hr_High_Chunk_Size",
    connection_args={"uri": URI},
)

vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="Hr_High_Chunk_Size",
)

uuids = [str(uuid4()) for _ in range(len(combined_docs))]
vector_store.add_documents(documents=combined_docs, ids=uuids)

In [ ]:
#This is from Code-Of_conduct
start_time = time.time()
results = vector_store.similarity_search(
    "What are the ethical business practices?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF CODE OF CONDUCT QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")
#This is from Compensation-Benefits Guide
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Medical Insurance & Surviving Spouse Coverage",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF COMPENSATIONS-BENEFITS GUIDE QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#This is from Employee Termination policy handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Voluntary Terminations",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE TERMINATION QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

#this is from employee handbook, but conflicts with compensation-benefits, so we have to see if it is captured correctly from employee handbook
start_time = time.time()
results = vector_store.similarity_search(
    "tell me about Compensation & development?",
    k=3,
)
print("QUERY EXECUTED IN --- %s seconds ---" % (time.time() - start_time))
for idx, res in enumerate(results):
    print(f"RESULTS OF EMPLOYEE HANDBOOK QUERY: {idx}")
    print(f"CONTENT: {res.page_content}")
    print(f"METADATA: {res.metadata}")
    print("== next response ==")

print("======================================NEXT QUERY==============================================")

## High Chunk Size Observations:



## Reflections:


## Final Decisions: